In [1]:
import pandas as pd
import numpy as np

# import seaborn as sns
# import matplotlib.pyplot as plt

## Pheno_merge

In [2]:
## load data
# phenotype
path = '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/phenotype_file/release_36K/'
ADNI = pd.read_csv(path + "ADNIPhenotypes_DS_2022.08.18_ALL.txt", sep='\t')
family_based = pd.read_csv(path + "ADSPFamilyBasedPhenotypes_DS_2022.08.18_ALL.txt", sep='\t',encoding='cp1252')
case_control = pd.read_csv(path + "ADSPCaseControlPhenotypes_DS_2022.08.18_ALL.txt", sep='\t')
PSPCBD = pd.read_csv(path + "PSPCBDPhenotypes_DS_2022.08.18_ALL.txt", sep='\t')
SUBJ_drop = pd.read_excel(path + 'ng00067_subject_droplist_2022.08.18.xlsx', engine = 'openpyxl')
PSPCBD_descript = pd.read_excel(path + 'PSPCBDPhenotypes_DD_2022.08.18.xlsx', engine = 'openpyxl')

#others
qc = pd.read_csv(path+"gcad.qc.r4.wgs.allchr.36361.GATK.2022.08.15.sample.summary.ALL.txt", sep = '\t') ## this one contains QC info( all the sampleID here are WGS)
ibd = pd.read_csv(path + 'gcad.r4.wgs.36361.2022.08.15.pairwise_IBD.txt', sep = '\t')

vcf = pd.read_csv(path + 'vcf_ids.txt', sep = '\t')
manifest = pd.read_csv(path + "SampleManifest_DS_2022.08.18_ALL.txt", sep='\t') ## the one with mapping info
duplist = pd.read_csv(path + 'duplist.csv') ## the lisan dup IBD list

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
## lisan's way
ADNI_processed = ADNI.loc[:,['SUBJID', 'Sex', 'PrevAD', 'IncAD','Age_current', 'Age_MCI_onset','Age_AD_onset', 'Age_baseline','APOE_reported','Race', 'AD_last_visit', 'Ethnicity']]
ADNI_processed["Diagnosis"] = "NaN"
ADNI_processed.loc[(ADNI_processed["PrevAD"] == 1) | (ADNI_processed["IncAD"] ==1),'Diagnosis'] =1
ADNI_processed.loc[(ADNI_processed["PrevAD"] == 0) & (ADNI_processed["IncAD"] ==0) &  (ADNI_processed["Age_MCI_onset"].isnull())& (ADNI_processed["Age_AD_onset"].isnull()),'Diagnosis'] =0

ADNI_processed["Age"] = ADNI_processed.Age_current
ADNI_processed.loc[ADNI_processed.Diagnosis == 1,'Age'] = ADNI_processed.loc[ADNI_processed.Diagnosis == 1,'Age_AD_onset']
#ADNI_processed.loc[ADNI_processed.Diagnosis == 0,'Age'] = ADNI_processed.loc[ADNI_processed.Diagnosis == 1,'Age_current']

ADNI_processed.shape

(1566, 14)

In [9]:
ADNI_processed.Diagnosis.value_counts()

1      611
NaN    525
0      430
Name: Diagnosis, dtype: int64

In [11]:
ADNI_processed.AD_last_visit.value_counts()

0    958
1    608
Name: AD_last_visit, dtype: int64

In [5]:
## check if there are the cases they defined that have AGE_MCI
ADNI_processed[(ADNI_processed.Diagnosis ==1) & (ADNI_processed.Age_MCI_onset.notnull())].shape

(137, 12)

In [123]:
print(ADNI_processed[ADNI_processed["AD_last_visit"] == 1].shape) 
print(ADNI_processed[ADNI_processed["AD_last_visit"] == 0].shape)

print(ADNI_processed[(ADNI_processed["AD_last_visit"] == 1) & (ADNI_processed["Age_MCI_onset"].notnull())].shape) ## case with MCI info
print(ADNI_processed[(ADNI_processed["AD_last_visit"] == 0) & (ADNI_processed["Age_MCI_onset"].notnull())].shape) ## control with MCI info
print(ADNI_processed[(ADNI_processed["AD_last_visit"] == 0) & (ADNI_processed["Age_MCI_onset"].isnull())].shape)

(608, 12)
(958, 12)
(134, 12)
(528, 12)
(430, 12)


In [128]:
## if remove those 528, will be the same as LiSan
ADNI_control[~ADNI_control.SUBJID.isin(ADNI_processed[(ADNI_processed["AD_last_visit"] == 0) & (ADNI_processed["Age_MCI_onset"].isnull())].SUBJID)]

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_MCI_onset,Age_AD_onset,Age_baseline,APOE_reported,Race,AD_last_visit,Diagnosis


In [130]:
## The four rows that is inconsistent with AD_last_visit and IncAD
ADNI_case[~ADNI_case.SUBJID.isin(ADNI_processed[ADNI_processed["AD_last_visit"] == 1].SUBJID)]

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_MCI_onset,Age_AD_onset,Age_baseline,APOE_reported,Race,AD_last_visit,Diagnosis
825,G-ADNI-AN000018,0,0,1,68,59,NaN,60,34,5.0,0,1
859,G-ADNI-AN000052,1,0,1,75,70,NaN,71,33,5.0,0,1
981,G-ADNI-AN000176,0,0,1,83,80,NaN,82,34,5.0,0,1
1276,G-ADNI-AN000478,0,0,1,66,63,NaN,62,33,5.0,0,1


In [134]:
## But there is a row that in their defination will should be removed but I think its a case (with AD_onset and AD_last_visit)
ADNI_processed[ADNI_processed["AD_last_visit"] == 1][~ADNI_processed[ADNI_processed["AD_last_visit"] == 1].SUBJID.isin(ADNI_case.SUBJID)]

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_MCI_onset,Age_AD_onset,Age_baseline,APOE_reported,Race,AD_last_visit,Diagnosis
231,ADNI_023_S_4115,0,0,0,75,64,75,67,33,5.0,1,NaN


In [ ]:
ADNI_control.SUBJID.isin(ADNI_processed[(ADNI_processed["AD_last_visit"] == 0) & (ADNI_processed["Age_MCI_onset"].isnull())].SUBJID)

In [15]:
ADNI_processed.head()

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_MCI_onset,Age_AD_onset,Age_baseline,APOE_reported,Race,AD_last_visit,Ethnicity,Diagnosis,Age
0,ADNI_002_S_0413,1,0,0,87,NaN,NaN,76,33,5.0,0,0.0,0,87
1,ADNI_002_S_0685,1,0,0,90+,NaN,NaN,89,33,5.0,0,0.0,0,90+
2,ADNI_002_S_0729,1,0,1,72,NaN,68,65,34,5.0,1,0.0,1,68
3,ADNI_002_S_1155,0,0,0,69,52,NaN,57,33,5.0,0,0.0,NaN,69
4,ADNI_002_S_1261,1,0,0,82,81,NaN,71,33,5.0,0,0.0,NaN,82


In [23]:
## family_based_lisan
## they are including 8.5k samples that doesnt have diagnosis
family_based = pd.read_csv(path + "ADSPFamilyBasedPhenotypes_DS_2022.08.18_ALL.txt", sep='\t',encoding='cp1252')
family_based['Diagnosis'] = 'NaN'

case = family_based[(family_based['AD'] >=1) & (family_based['AD'] <=3)].copy()
control = family_based[family_based['AD'] == 0].copy()
print('lisan\n case:',case.shape, ', control:',control.shape)

control.loc[:,["Diagnosis"]] = 0
case.loc[:,["Diagnosis"]] = 1
#family_based = pd.concat([control,case])
#family_based_processed = family_based.loc[:, ["SUBJID", "Sex", "Diagnosis", "APOE_reported","Age","Race", "Age_baseline"]]

family_based.loc[(family_based['AD'] >=1) & (family_based['AD'] <=3),'Diagnosis'] = 1
family_based.loc[family_based['AD'] ==0,'Diagnosis'] = 0
family_based_processed = family_based

lisan
 case: (1858, 28) , control: (2493, 28)


In [24]:
family_based_processed = family_based
family_based_processed.Diagnosis.value_counts()

NaN    8507
0      2493
1      1858
Name: Diagnosis, dtype: int64

In [25]:
## case_control
case_control_processed = case_control.loc[:,["SUBJID", "Sex", "AD", "APOE_reported","Age","Race", "Age_baseline",'Ethnicity']]
case_control.AD = case_control.AD.astype('Int64')
case_control_processed = case_control_processed.rename(columns = {'AD':'Diagnosis'})
case_control_processed.Diagnosis.value_counts(dropna=False)


0.0    23347
1.0    16793
NaN     5235
Name: Diagnosis, dtype: int64

In [26]:
ADNI_processed["source"] = 'ADNI'
family_based_processed["source"] = 'Family_based'
case_control_processed["source"] = 'case_control'

In [27]:
pheno = pd.concat([ADNI_processed, family_based_processed, case_control_processed])
pheno.loc[pheno['Age'] == '90+', 'Age'] = 90
pheno.loc[pheno['Age_baseline'] == '90+', 'Age_baseline'] = 90


print(pheno.shape)
print(pheno.Diagnosis.value_counts())
#pheno = pheno.loc[:,['SUBJID','Sex','Age_AD_onset','Age_MCI_onset','Age_baseline','APOE_reported','Race','Diagnosis','source','Ethnicity']]
print(pheno.shape)

(59799, 35)
0      26270
1      19262
NaN     9032
Name: Diagnosis, dtype: int64
(59799, 35)


## deal with duplication

In [28]:
wgs_SUBJID = manifest[manifest.SampleID.isin(qc.SampleID)].SUBJID
wgs_SUBJID.shape

(36361,)

In [38]:
print(manifest[(manifest.SUBJID.isin(wgs_SUBJID)) & manifest.SAMPLE_USE == 'WES'])
print(manifest.SAMPLE_USE.value_counts())

Empty DataFrame
Columns: [SUBJID, SampleID, BODY_SITE, ANALYTE_TYPE, Sequencing_Center, Sequencing_Platform, SAMPLE_USE, Technical_Replicate, Study_DSS, Sample_Set]
Index: []
WGS    36361
WES    20502
Name: SAMPLE_USE, dtype: int64


In [29]:
print('before extracting WGS', pheno.shape)
pheno = pheno[pheno.SUBJID.isin(wgs_SUBJID)]

print('After extracting WGS', pheno.shape)

before extracting WGS (59799, 35)
After extracting WGS (33639, 35)


In [30]:
## overlaps in Family and Case control, keep the one in Family
print(pheno[(pheno.SUBJID.isin(pheno[pheno.SUBJID.duplicated()].SUBJID)) & (pheno.source == 'case_control')].shape)
pheno[pheno.SUBJID.duplicated()]
pheno = pheno[~pheno.SUBJID.duplicated()] ## only keep family study like they suggest

(32, 35)


In [31]:
## drop technique replicates (using subjid)
replicate_to_drop = manifest[(manifest.SampleID.isin(qc.SampleID)) & (manifest.Technical_Replicate == 1)] ##(63,10)
pheno[pheno.SUBJID.isin(replicate_to_drop.SUBJID)]
pheno = pheno[~pheno.SUBJID.isin(replicate_to_drop.SUBJID)]
#pheno.to_csv(path + 'check_pheno.tsv',sep = '\t', index = False)
pheno.shape

(33601, 35)

## QC sample (deal with the not 1-1 matching in pheno file)
merge sampleID with SUBJID using manifest file. for those SUBJID that have multiple SampleIDs, pick the one with the lowest missing rate (qc file)

In [35]:
pheno_add_SampleID  = pheno.merge(manifest, on="SUBJID")
print(pheno_add_SampleID.shape)
dup_sample = pheno_add_SampleID[pheno_add_SampleID.SUBJID.duplicated(keep=False)] #9033 row


(33601, 44)
Series([], dtype: int64)


(0,)

In [41]:
dup_SampleID = manifest[manifest.SUBJID.isin(dup_sample.SUBJID.unique())].SampleID ##9033
print(qc[qc.SampleID.isin(dup_SampleID)].Missing.idxmin())
dup_SampleID[~dup_SampleID.isin(qc.SampleID)]  ## not every SampleID is found in QC file

22552


4            A-ACT-AC000014-BL-UPN-23967
13           A-ACT-AC000034-BL-UPN-15865
26           A-ACT-AC000057-BL-UPN-14863
31            A-ACT-AC000072-BL-UPN-7532
38           A-ACT-AC000088-BL-UPN-13977
                      ...               
20489    A-ADC-AD000841-BL-NCR-09AD16180
20490    A-ADC-AD000842-BL-NCR-09AD16177
20491    A-ADC-AD000862-BL-NCR-09AD16175
20492    A-ADC-AD001075-BL-NCR-09AD16172
20493    A-ADC-AD001332-BL-NCR-09AD16171
Name: SampleID, Length: 3831, dtype: object

In [64]:
## extract the SampleID with the lowest missing num
best_SampleID=[]
for i in dup_sample.SUBJID.unique():
    SampleID = manifest[manifest.SUBJID==i].SampleID
    best_id = qc[qc.SampleID.isin(SampleID)].Missing.idxmin()
    best_SampleID.append(qc.loc[best_id].SampleID)
    

In [65]:
dup_remove = dup_sample[~dup_sample.SampleID.isin(best_SampleID)]
print(dup_remove.shape) ## 4532 rows to remove
pheno_add_SampleID = pheno_add_SampleID[~pheno_add_SampleID.SampleID.isin(dup_remove.SampleID)] ## remove 4532 rows, 33,601 rows remain

pheno_add_SampleID.to_csv(path + 'pheno_SampleID.tsv',sep = '\t', index = False)

(4532, 44)


## Deal with non-concordant phenotype based on APOE status and AD

In [3]:
pheno_add_SampleID = pd.read_csv(path + 'pheno_SampleID.tsv',sep = '\t')

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,6,15,16,17,23,24,25,27,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
### 1. Deal with IBD
## mapping SampleID to SUBJID
ibdpair_95 = ibd[ibd.PI_HAT>.95]
mapping = manifest.loc[(manifest['SampleID'].isin(ibdpair_95.SampleID1)) | (manifest['SampleID'].isin(ibdpair_95.SampleID2)),['SUBJID','SampleID']]
map_dict = dict(list(zip(mapping.SampleID, mapping.SUBJID)))

ibdpair_95[ibdpair_95.SampleID1.isin(manifest.SampleID)] ## all are found

## Add SUBJID to IBD pair
ibdpair_95.loc[:,['SUBJID1']] = ibdpair_95['SampleID1'].map(map_dict)
ibdpair_95.loc[:,['SUBJID2']] = ibdpair_95['SampleID2'].map(map_dict)
ibdpair_95

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2
3217,A-LOAD-LD004376-BL-NCR-04AD6509,A-LOAD-LD004376-UNK-UNK-UNK0036,0.9834,A-LOAD-LD004376,A-LOAD-LD004376
3218,A-ADC-AD008841-BL-NCR-10AD24304,G-KGAD-KA000206-UNK-WU-8005874212,0.9897,A-ADC-AD008841,G-KGAD-KA000206
3219,A-ADC-AD016717-BL-NCR-13AD63111,A-ADC-AD016717-BL-NCR-13AD63111uf,0.9914,A-ADC-AD016717,A-ADC-AD016717
3220,A-LOAD-LD004379-BL-NCR-08AD9443,A-LOAD-LD004379-UNK-UNK-UNK0037,0.9924,A-LOAD-LD004379,A-LOAD-LD004379
3221,A-CUHS-CU009833-BL-COL-55252BL1,A-CUHS-CU009833-BL-COL-55252BL1ld,0.9927,A-CUHS-CU009833,A-CUHS-CU009833
...,...,...,...,...,...
4811,P-UCLA-LA000059-UNK-UPN-F03,P-UCSF-PSP10538-UNK-UPN-8882,1.0000,P-UCLA-LA000059,P-UCSF-PSP10538
4812,P-UCLA-LA000137-UNK-UPN-E03,P-UPN-PSP10085-BL-UPN-112194,1.0000,P-UCLA-LA000137,P-UPN-PSP10085
4813,P-UCLA-LA000207-UNK-UPN-B11,P-UPN-PSP10065-BL-UPN-104130,1.0000,P-UCLA-LA000207,P-UPN-PSP10065
4814,P-UCLA-LA000394-UNK-UPN-1000016,P-UCSF-PSP10540-UNK-UPN-10886,1.0000,P-UCLA-LA000394,P-UCSF-PSP10540


In [5]:
## these are not duplicates from the same SUBJECT but the pi_hat is high 
dup_ibd = ibdpair_95[ibdpair_95.SUBJID1 != ibdpair_95.SUBJID2] ## 513
dup_ibd = dup_ibd[(dup_ibd.SUBJID1.isin(pheno_add_SampleID.SUBJID)) & (dup_ibd.SUBJID2.isin(pheno_add_SampleID.SUBJID))] #458 (They used & instead of | here)

In [6]:
## only deal with the columns that they handle manually
dup_ibd_merge = dup_ibd.merge(pheno_add_SampleID[['SUBJID','Diagnosis','Race','APOE_reported','Ethnicity','Sex','Age']], left_on = 'SUBJID1',right_on = 'SUBJID')
dup_ibd_merge = dup_ibd_merge.merge(pheno_add_SampleID[['SUBJID','Diagnosis','Race','APOE_reported','Ethnicity','Sex','Age']], left_on = 'SUBJID2',right_on = 'SUBJID')

In [8]:
## Remove the rows that inconsistent with AD diagnosis or APOE status ## 136 rows in total
inconsistent = dup_ibd_merge[(dup_ibd_merge.Diagnosis_x != dup_ibd_merge.Diagnosis_y) | 
                            (dup_ibd_merge.APOE_reported_x != dup_ibd_merge.APOE_reported_y)]
dup_ibd_merge = pd.concat([dup_ibd_merge,inconsistent]).drop_duplicates(keep=False) ## only keep the dup_ibd_merge not in inconsistent

dup_ibd_merge.shape

(320, 19)

In [9]:
check_inconsistent = duplist[duplist.Choice == 'Inconsistent']
inconsistent[~inconsistent.SampleID1.isin(check_inconsistent.SampleID1)]

,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2,SUBJID_x,Diagnosis_x,Race_x,APOE_reported_x,Ethnicity_x,Sex_x,Age_x,SUBJID_y,Diagnosis_y,Race_y,APOE_reported_y,Ethnicity_y,Sex_y,Age_y
155,A-ADC-AD016640-BL-NCR-14AD70496uf,A-WCAP-WC001612-BL-COL-49158BL1,1.0,A-ADC-AD016640,A-WCAP-WC001612,A-ADC-AD016640,NaN,6.0,33.0,1.0,0.0,NaN,A-WCAP-WC001612,1.0,6.0,33.0,1.0,0.0,72.0
387,A-ADC-AD015732-BL-NCR-17AD90732e,A-ADC-AD015740-BL-NCR-18AD92097,1.0,A-ADC-AD015732,A-ADC-AD015740,A-ADC-AD015732,NaN,5.0,33.0,0.0,0.0,NaN,A-ADC-AD015740,NaN,5.0,33.0,0.0,0.0,NaN


In [133]:
## Check the age 
dup_ibd_merge[(dup_ibd_merge.Age_x.isnull()) & (dup_ibd_merge.Age_y.isnull())]

,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2,SUBJID_x,Diagnosis_x,Race_x,APOE_reported_x,Ethnicity_x,Sex_x,Age_x,SUBJID_y,Diagnosis_y,Race_y,APOE_reported_y,Ethnicity_y,Sex_y,Age_y


In [11]:
## chose the older age, if equal, choose the first one
dup_ibd_merge[dup_ibd_merge.Age_x == dup_ibd_merge.Age_y].shape
dup_ibd_merge['Age'] = dup_ibd_merge[["Age_x","Age_y"]].apply(max,axis = 1)

dup_ibd_merge.loc[dup_ibd_merge.Age_x >= dup_ibd_merge.Age_y,'SUBJID']=dup_ibd_merge.SUBJID1
dup_ibd_merge.loc[dup_ibd_merge.Age_x < dup_ibd_merge.Age_y,'SUBJID']=dup_ibd_merge.SUBJID2

dup_ibd_merge.loc[dup_ibd_merge.Age_x >= dup_ibd_merge.Age_y,'SampleID']=dup_ibd_merge.SampleID1
dup_ibd_merge.loc[dup_ibd_merge.Age_x < dup_ibd_merge.Age_y,'SampleID']=dup_ibd_merge.SampleID2

dup_ibd_merge.loc[:,['SUBJID1','SUBJID2','Age_x','Age_y','SUBJID','SampleID']]


,SUBJID1,SUBJID2,Age_x,Age_y,SUBJID,SampleID
0,A-ADC-AD008841,G-KGAD-KA000206,54.0,55.0,G-KGAD-KA000206,G-KGAD-KA000206-UNK-WU-8005874212
1,A-ADC-AD010154,ADNI_011_S_0023,81.0,79.0,A-ADC-AD010154,A-ADC-AD010154-BL-NCR-13AD63381
2,A-MIA-UM011643,ADNI_011_S_0023,85.0,79.0,A-MIA-UM011643,A-MIA-UM011643-BL-MIA-202014718
3,A-ADC-AD010154,A-MIA-UM011643,81.0,85.0,A-MIA-UM011643,A-MIA-UM011643-BL-MIA-202014718
4,A-ADC-AD005494,A-VAN-VU001340,65.0,65.0,A-ADC-AD005494,A-ADC-AD005494-BL-NCR-09AD17556uf
...,...,...,...,...,...,...
452,A-NMAS-NM000498,A-WCAP-WC001585,80.0,81.0,A-WCAP-WC001585,A-WCAP-WC001585-BL-COL-45991BL1
453,A-NMAS-NM000567,A-WCAP-WC001376,77.0,78.0,A-WCAP-WC001376,A-WCAP-WC001376-BL-COL-39666BL1
455,A-NMAS-NM000756,A-WCAP-WC001622,75.0,76.0,A-WCAP-WC001622,A-WCAP-WC001622-BL-COL-50921BL1
456,A-NMAS-NM000760,A-WCAP-WC001573,76.0,76.0,A-NMAS-NM000760,A-NMAS-NM000760-BL-MIA-200810732


In [19]:
dup_ibd_merge.shape

(320, 22)

In [27]:
##diff chose 1
dup_ibd_merge[dup_ibd_merge.SUBJID==dup_ibd_merge.SUBJID1][~dup_ibd_merge[dup_ibd_merge.SUBJID==dup_ibd_merge.SUBJID1].SUBJID.isin(duplist[duplist.Choice =='Select1'].SUBJID1)][['SampleID1','SampleID2','Age_x','Age_y','Diagnosis_x','SampleID']]

,SampleID1,SampleID2,Age_x,Age_y,Diagnosis_x,SampleID
2,A-MIA-UM011643-BL-MIA-202014718,ADNI_011_S_0023,85.0,79.0,0.0,A-MIA-UM011643-BL-MIA-202014718
86,A-ADC-AD006858-BL-NCR-09AD16022e,G-ADNI-AN000667-BL-NCR-08AD6948,66.0,66.0,1.0,A-ADC-AD006858-BL-NCR-09AD16022e


In [29]:
dup_ibd_merge[dup_ibd_merge.SampleID1 == 'A-ADC-AD006858-BL-NCR-09AD16022e'][['SampleID1','SampleID2','Age_x','Age_y','Diagnosis_x','SampleID']]

,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2,Choice
81,A-ADC-AD006858-BL-NCR-09AD16022e,A-LOAD-LD001253-UNK-UNK-UNK0017,0.9999,A-ADC-AD006858,A-LOAD-LD001253,Select2
217,A-ADC-AD006858-BL-NCR-09AD16022e,G-ADNI-AN000667-BL-NCR-08AD6948,1.0000,A-ADC-AD006858,G-ADNI-AN000667,PrunedEarlier1


In [32]:
duplist[duplist.SampleID1 == 'A-ADC-AD006858-BL-NCR-09AD16022e']

,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2,Choice
81,A-ADC-AD006858-BL-NCR-09AD16022e,A-LOAD-LD001253-UNK-UNK-UNK0017,0.9999,A-ADC-AD006858,A-LOAD-LD001253,Select2
217,A-ADC-AD006858-BL-NCR-09AD16022e,G-ADNI-AN000667-BL-NCR-08AD6948,1.0000,A-ADC-AD006858,G-ADNI-AN000667,PrunedEarlier1


In [17]:
duplist[duplist.Choice =='Select2']

,SampleID1,SampleID2,PI_HAT,SUBJID1,SUBJID2,Choice
0,A-ADC-AD008841-BL-NCR-10AD24304,G-KGAD-KA000206-UNK-WU-8005874212,0.9897,A-ADC-AD008841,G-KGAD-KA000206,Select2
2,A-ADC-AD010154-BL-NCR-13AD63381,A-MIA-UM011643-BL-MIA-202014718,0.9953,A-ADC-AD010154,A-MIA-UM011643,Select2
8,A-ADC-AD013256-BL-NCR-13AD61079uf,G-KGAD-KA000037-UNK-WU-8005872491,0.9985,A-ADC-AD013256,G-KGAD-KA000037,Select2
16,A-ADC-AD007007-BL-NCR-10AD30408,G-KGAD-KA000593-UNK-WU-8005871830,0.9990,A-ADC-AD007007,G-KGAD-KA000593,Select2
19,A-ADC-AD005555-BL-NCR-09AD16178,A-LOAD-LD001742-BL-NCR-07AD2155,0.9991,A-ADC-AD005555,A-LOAD-LD001742,Select2
...,...,...,...,...,...,...
446,A-NMAS-NM000342-BL-MIA-200811001,A-WCAP-WC001298-BL-COL-50945BL1,1.0000,A-NMAS-NM000342,A-WCAP-WC001298,Select2
448,A-NMAS-NM000389-BL-MIA-200810312,A-WCAP-WC001226-BL-COL-38633BL1,1.0000,A-NMAS-NM000389,A-WCAP-WC001226,Select2
452,A-NMAS-NM000498-BL-MIA-200810742,A-WCAP-WC001585-BL-COL-45991BL1,1.0000,A-NMAS-NM000498,A-WCAP-WC001585,Select2
453,A-NMAS-NM000567-BL-MIA-200810281,A-WCAP-WC001376-BL-COL-39666BL1,1.0000,A-NMAS-NM000567,A-WCAP-WC001376,Select2


In [14]:
print(dup_ibd_merge[dup_ibd_merge.SUBJID==dup_ibd_merge.SUBJID1].shape) 
print(dup_ibd_merge[dup_ibd_merge.SUBJID==dup_ibd_merge.SUBJID2].shape)

(151, 22)
(153, 22)


In [12]:
dup_ibd_merge = dup_ibd_merge.rename(columns = {'Sex_x':'Sex','Ethnicity_x':'Ethnicity','Race_x':'Race'})
print(dup_ibd_merge[['Sex','Ethnicity','Race']].isnull().sum())
dup_ibd_merge.Ethnicity =dup_ibd_merge.Ethnicity.combine_first(dup_ibd_merge.Ethnicity_y)


Sex          0
Ethnicity    1
Race         0
dtype: int64


In [ ]:
pheno_IBD = pheno_add_SampleID.loc[pheno_add_SampleID.SampleID.isin(dup_ibd_pheno.SampleID1) | pheno_add_SampleID.SampleID.isin(dup_ibd_pheno.SampleID2)]
dup_ibd_pheno.shape

In [85]:
# Replace 'col' with 'column' in all column names
dup_ibd_pheno = dup_ibd_pheno.drop(columns=['Age_x','SUBJID_x','SampleID_x'])
dup_ibd_pheno = dup_ibd_pheno.rename(columns={col: col.replace('_x', '') for col in dup_ibd_pheno.columns})
ibd_to_keep = dup_ibd_pheno[pheno_IBD.columns]
print(pheno_IBD.shape)

(614, 44)


In [92]:
pheno_add_SampleID[(~pheno_add_SampleID.SampleID.isin(dup_ibd.SampleID1)) & (~pheno_add_SampleID.SampleID.isin(dup_ibd.SampleID2))].shape

(32708, 44)

In [93]:
test = pd.concat([pheno_add_SampleID[(~pheno_add_SampleID.SampleID.isin(dup_ibd.SampleID1)) & (~pheno_add_SampleID.SampleID.isin(dup_ibd.SampleID2))], ibd_to_keep])

In [97]:
pheno_add_SampleID.shape

(33601, 44)

In [96]:
test.to_csv(path + 'pheno_concordant.tsv',sep = '\t', index = False)

In [98]:
pheno_add_SampleID.to_csv(path + 'pheno_SampleID.tsv',sep = '\t', index = False)

In [55]:
pheno_IBD.SampleID = dup_ibd_pheno.SampleID

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [109]:
pheno_IBD.loc[:,'SampleID']

0        G-KGAD-KA000206-UNK-WU-8005874212
2          A-MIA-UM011643-BL-MIA-202014718
5                                      NaN
6          A-ADC-AD002931-BL-NCR-09AD17996
11                                     NaN
                       ...                
35097                                  NaN
35106                                  NaN
37806                                  NaN
38073                                  NaN
38078                                  NaN
Name: SampleID, Length: 614, dtype: object

In [96]:
pheno_add_SampleID.loc[pheno_add_SampleID.SampleID.isin(dup_ibd_pheno.SampleID1) | pheno_add_SampleID.SampleID.isin(dup_ibd_pheno.SampleID2),'SampleID'] = dup_ibd_pheno.SampleID